In [2]:
#we have the environment, let's test to see if enformer is here

import tensorflow as tf
import tensorflow_hub as hub

enformer_model = hub.load("https://kaggle.com/models/deepmind/enformer/frameworks/TensorFlow2/variations/enformer/versions/1").model

SEQ_LENGTH = 393_216

# Numpy array [batch_size, SEQ_LENGTH, 4] one hot encoded in order 'ACGT'. The
# `one_hot_encode` function is available in `enformer.py` and outputs can be
# stacked to form a batch.

2024-07-02 15:34:12.261916: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcuda.so.1
2024-07-02 15:34:12.413880: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1733] Found device 0 with properties: 
pciBusID: 0000:02:00.0 name: NVIDIA GeForce GTX 1080 Ti computeCapability: 6.1
coreClock: 1.582GHz coreCount: 28 deviceMemorySize: 10.91GiB deviceMemoryBandwidth: 451.17GiB/s
2024-07-02 15:34:12.414501: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1733] Found device 1 with properties: 
pciBusID: 0000:03:00.0 name: NVIDIA GeForce GTX 1080 Ti computeCapability: 6.1
coreClock: 1.582GHz coreCount: 28 deviceMemorySize: 10.91GiB deviceMemoryBandwidth: 451.17GiB/s
2024-07-02 15:34:12.415063: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1733] Found device 2 with properties: 
pciBusID: 0000:83:00.0 name: NVIDIA GeForce GTX 1080 Ti computeCapability: 6.1
coreClock: 1.582GHz coreCount: 28 deviceMemorySize: 10.91GiB deviceMemoryBan

NameError: name 'enformer' is not defined

In [4]:
enformer_model

<tensorflow.python.saved_model.load.Loader._recreate_base_user_object.<locals>._UserObject at 0x7f077819fc10>

In [5]:
inputs = tf.zeros((1, SEQ_LENGTH, 4), dtype=tf.float32)
predictions = enformer_model.predict_on_batch(inputs)
predictions['human'].shape

2024-07-02 15:34:42.594829: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:176] None of the MLIR Optimization Passes are enabled (registered 2)
2024-07-02 15:34:42.766977: I tensorflow/core/platform/profile_utils/cpu_utils.cc:114] CPU Frequency: 2299875000 Hz


TensorShape([1, 896, 5313])

In [7]:
a = predictions['human'][0,:,27] #for the hepg2 cell line, not real data tho, all N

In [8]:
a.shape

TensorShape([896])

In [15]:
#check out the attributes
enformer_model

<tensorflow.python.saved_model.load.Loader._recreate_base_user_object.<locals>._UserObject at 0x7f077819fc10>

In [18]:
dir(enformer_model)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_add_variable_with_custom_getter',
 '_checkpoint_dependencies',
 '_crop_final',
 '_crop_input',
 '_deferred_dependencies',
 '_delete_tracking',
 '_downres',
 '_final_pointwise',
 '_gather_saveables_for_checkpoint',
 '_handle_deferred_dependencies',
 '_heads',
 '_list_extra_dependencies_for_serialization',
 '_list_functions_for_serialization',
 '_lookup_dependency',
 '_map_resources',
 '_maybe_initialize_trackable',
 '_name_based_attribute_restore',
 '_name_based_restores',
 '_no_dependency',
 '_object_identifier',
 '_preload_simple_restoration',
 '_restore_from_checkpoint_position',
 '_self_name_based_restores',
 '_self_

In [19]:
dir(enformer_model.predict_on_batch)

['__call__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__get__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__name__',
 '__ne__',
 '__new__',
 '__original_wrapped__',
 '__qualname__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '__wrapped__',
 '_autograph',
 '_call',
 '_clone',
 '_concrete_stateful_fn',
 '_create_implements_attribute',
 '_created_variables',
 '_decorate',
 '_defun',
 '_defun_with_scope',
 '_descriptor_cache',
 '_experimental_autograph_options',
 '_experimental_follow_type_hints',
 '_experimental_relax_shapes',
 '_function_spec',
 '_get_concrete_function_garbage_collected',
 '_get_key_for_call_stats',
 '_get_tracing_count',
 '_graph_deleter',
 '_implements',
 '_initialize',
 '_initialize_uninitialized_variables',
 '_i

In [ ]:
#eh it's fine, the point is that we can use it to predict just fine!

# next step is to convert the data, we have all these TFR files!

In [1]:
import glob
import json
import functools
import os
import tensorflow as tf
import numpy as np
import pandas as pd

def organism_path(organism):
  return os.path.join('/data/leslie/sarthak/data/enformer/data/', organism)


def get_dataset(organism, subset, num_threads=8):
  metadata = get_metadata(organism)
  dataset = tf.data.TFRecordDataset(tfrecord_files(organism, subset),
                                    compression_type='ZLIB',
                                    num_parallel_reads=num_threads)
  dataset = dataset.map(functools.partial(deserialize, metadata=metadata),
                        num_parallel_calls=num_threads)
  return dataset


def get_metadata(organism):
  # Keys:
  # num_targets, train_seqs, valid_seqs, test_seqs, seq_length,
  # pool_width, crop_bp, target_length
  path = os.path.join(organism_path(organism), 'statistics.json')
  with tf.io.gfile.GFile(path, 'r') as f:
    return json.load(f)


def tfrecord_files(organism, subset):
  # Sort the values by int(*).
  return sorted(tf.io.gfile.glob(os.path.join(
      organism_path(organism), 'tfrecords', f'{subset}-*.tfr'
  )), key=lambda x: int(x.split('-')[-1].split('.')[0]))


def deserialize(serialized_example, metadata):
  """Deserialize bytes stored in TFRecordFile."""
  feature_map = {
      'sequence': tf.io.FixedLenFeature([], tf.string),
      'target': tf.io.FixedLenFeature([], tf.string),
  }
  example = tf.io.parse_example(serialized_example, feature_map)
  sequence = tf.io.decode_raw(example['sequence'], tf.bool)
  sequence = tf.reshape(sequence, (metadata['seq_length'], 4))
  sequence = tf.cast(sequence, tf.float32)

  target = tf.io.decode_raw(example['target'], tf.float16)
  target = tf.reshape(target,
                      (metadata['target_length'], metadata['num_targets']))
  target = tf.cast(target, tf.float32)

  return {'sequence': sequence,
          'target': target}

def get_targets(organism):
  targets_txt = f'https://raw.githubusercontent.com/calico/basenji/master/manuscripts/cross2020/targets_{organism}.txt'
  return pd.read_csv(targets_txt, sep='\t')


2024-07-03 18:57:02.467801: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0


In [2]:
import pandas as pd
def get_targets(organism):
  targets_txt = f'https://raw.githubusercontent.com/calico/basenji/master/manuscripts/cross2020/targets_{organism}.txt'
  return pd.read_csv(targets_txt, sep='\t')

In [3]:
df_targets_human = get_targets('human')
df_targets_human.head()

,index,genome,identifier,file,clip,scale,sum_stat,description
0,0,0,ENCFF833POA,/home/drk/tillage/datasets/human/dnase/encode/...,32,2,mean,DNASE:cerebellum male adult (27 years) and mal...
1,1,0,ENCFF110QGM,/home/drk/tillage/datasets/human/dnase/encode/...,32,2,mean,DNASE:frontal cortex male adult (27 years) and...
2,2,0,ENCFF880MKD,/home/drk/tillage/datasets/human/dnase/encode/...,32,2,mean,DNASE:chorion
3,3,0,ENCFF463ZLQ,/home/drk/tillage/datasets/human/dnase/encode/...,32,2,mean,DNASE:Ishikawa treated with 0.02% dimethyl sul...
4,4,0,ENCFF890OGQ,/home/drk/tillage/datasets/human/dnase/encode/...,32,2,mean,DNASE:GM03348


In [6]:
human_dataset = get_dataset('human', 'train').batch(1).repeat()

2024-07-03 18:58:16.077749: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcuda.so.1
2024-07-03 18:58:16.171477: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1733] Found device 0 with properties: 
pciBusID: 0000:02:00.0 name: NVIDIA GeForce GTX 1080 Ti computeCapability: 6.1
coreClock: 1.582GHz coreCount: 28 deviceMemorySize: 10.91GiB deviceMemoryBandwidth: 451.17GiB/s
2024-07-03 18:58:16.171965: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1733] Found device 1 with properties: 
pciBusID: 0000:03:00.0 name: NVIDIA GeForce GTX 1080 Ti computeCapability: 6.1
coreClock: 1.582GHz coreCount: 28 deviceMemorySize: 10.91GiB deviceMemoryBandwidth: 451.17GiB/s
2024-07-03 18:58:16.172536: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1733] Found device 2 with properties: 
pciBusID: 0000:83:00.0 name: NVIDIA GeForce GTX 1080 Ti computeCapability: 6.1
coreClock: 1.582GHz coreCount: 28 deviceMemorySize: 10.91GiB deviceMemoryBan

In [9]:
human_dataset

<RepeatDataset shapes: {sequence: (None, 131072, 4), target: (None, 896, 5313)}, types: {sequence: tf.float32, target: tf.float32}>

In [10]:
it = iter(human_dataset)
example = next(it)

2024-07-03 19:00:02.388551: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:176] None of the MLIR Optimization Passes are enabled (registered 2)
2024-07-03 19:00:02.390178: I tensorflow/core/platform/profile_utils/cpu_utils.cc:114] CPU Frequency: 2299880000 Hz


In [14]:
example

{'sequence': <tf.Tensor: shape=(1, 131072, 4), dtype=float32, numpy=
 array([[[0., 0., 0., 1.],
         [0., 0., 1., 0.],
         [1., 0., 0., 0.],
         ...,
         [0., 0., 0., 1.],
         [1., 0., 0., 0.],
         [0., 0., 0., 1.]]], dtype=float32)>,
 'target': <tf.Tensor: shape=(1, 896, 5313), dtype=float32, numpy=
 array([[[0.10839844, 0.10760498, 0.04425049, ..., 0.        ,
          0.        , 0.        ],
         [0.10162354, 0.09332275, 0.0094986 , ..., 0.        ,
          0.        , 0.        ],
         [0.10272217, 0.15600586, 0.        , ..., 0.        ,
          0.        , 0.        ],
         ...,
         [0.07714844, 0.07678223, 0.03509521, ..., 0.        ,
          0.01934814, 0.        ],
         [0.07666016, 0.03826904, 0.05648804, ..., 0.        ,
          0.        , 0.        ],
         [0.08319092, 0.06051636, 0.02156067, ..., 0.        ,
          0.        , 0.        ]]], dtype=float32)>}

In [17]:
example['sequence'] #it's the tf tensor, so we have officially accessed one of these?

<tf.Tensor: shape=(1, 131072, 4), dtype=float32, numpy=
array([[[0., 0., 0., 1.],
        [0., 0., 1., 0.],
        [1., 0., 0., 0.],
        ...,
        [0., 0., 0., 1.],
        [1., 0., 0., 0.],
        [0., 0., 0., 1.]]], dtype=float32)>

In [18]:
example2 = next(it)
print(example2) #wait this is the next example, still part of the same TFR file?

{'sequence': <tf.Tensor: shape=(1, 131072, 4), dtype=float32, numpy=
array([[[0., 0., 0., 1.],
        [0., 0., 0., 1.],
        [1., 0., 0., 0.],
        ...,
        [0., 0., 0., 1.],
        [0., 0., 0., 1.],
        [1., 0., 0., 0.]]], dtype=float32)>, 'target': <tf.Tensor: shape=(1, 896, 5313), dtype=float32, numpy=
array([[[0.07519531, 0.01112366, 0.01609802, ..., 0.        ,
         0.        , 0.        ],
        [0.0881958 , 0.01392365, 0.03829956, ..., 0.        ,
         0.        , 0.        ],
        [0.05108643, 0.01622009, 0.01737976, ..., 0.        ,
         0.        , 0.        ],
        ...,
        [0.00142574, 0.001544  , 0.        , ..., 0.        ,
         0.        , 0.        ],
        [0.00675964, 0.        , 0.        , ..., 0.        ,
         0.        , 0.        ],
        [0.05609131, 0.0559082 , 0.06762695, ..., 0.        ,
         0.        , 0.        ]]], dtype=float32)>}


In [19]:
#the next question is whether this gives us a single TFR file or goes through all of them?
print(896*5313+131072*4)

5284736


In [20]:
temp_array=np.zeros((896*5313+131072*4)).astype(np.float32)
#see the size in GB
temp_array.nbytes/1e9
#yeah so the storage is the fact that it's many batches!
#we have to figure out what the function is doing

0.021138944

In [26]:
print(human_dataset)
print(len(human_dataset)) #when it was defined with .repeat() it's infinite


<RepeatDataset shapes: {sequence: (None, 131072, 4), target: (None, 896, 5313)}, types: {sequence: tf.float32, target: tf.float32}>


TypeError: dataset length is infinite.

In [24]:
test_dataset = get_dataset('human', 'test').batch(1)


TypeError: dataset length is unknown.

# to dos

Figure out how to get all the separate elements of the dataset, and then make it so that we can store this as some sort of np array!

Test the loss and see how we can recreate this in pytorch with some known values! 
 
loss = tf.reduce_mean(
          tf.keras.losses.poisson(batch['target'], outputs))

In [1]:
import glob
import json
import functools
import os
import tensorflow as tf
import numpy as np
import pandas as pd

def organism_path(organism):
  return os.path.join('/data/leslie/sarthak/data/enformer/data/', organism) #just leads to the organism path, quite easy


def get_dataset(organism, subset, num_threads=8): #this gets the data and deserializes it
  metadata = get_metadata(organism)
  dataset = tf.data.TFRecordDataset(tfrecord_files(organism, subset),
                                    compression_type='ZLIB',
                                    num_parallel_reads=num_threads)
  dataset = dataset.map(functools.partial(deserialize, metadata=metadata),
                        num_parallel_calls=num_threads)
  return dataset


def get_metadata(organism):
  # Keys:
  # num_targets, train_seqs, valid_seqs, test_seqs, seq_length,
  # pool_width, crop_bp, target_length
  path = os.path.join(organism_path(organism), 'statistics.json')
  with tf.io.gfile.GFile(path, 'r') as f:
    return json.load(f)


def tfrecord_files(organism, subset): #this is the function that gets the TFR files, gets all of them and sorts them
  # Sort the values by int(*).
  return sorted(tf.io.gfile.glob(os.path.join(
      organism_path(organism), 'tfrecords', f'{subset}-*.tfr'
  )), key=lambda x: int(x.split('-')[-1].split('.')[0]))


def deserialize(serialized_example, metadata):
  """Deserialize bytes stored in TFRecordFile."""
  feature_map = {
      'sequence': tf.io.FixedLenFeature([], tf.string),
      'target': tf.io.FixedLenFeature([], tf.string),
  }
  example = tf.io.parse_example(serialized_example, feature_map)
  sequence = tf.io.decode_raw(example['sequence'], tf.bool)
  sequence = tf.reshape(sequence, (metadata['seq_length'], 4))
  sequence = tf.cast(sequence, tf.float32)

  target = tf.io.decode_raw(example['target'], tf.float16)
  target = tf.reshape(target,
                      (metadata['target_length'], metadata['num_targets']))
  target = tf.cast(target, tf.float32)

  return {'sequence': sequence,
          'target': target}

def get_targets(organism):
  targets_txt = f'https://raw.githubusercontent.com/calico/basenji/master/manuscripts/cross2020/targets_{organism}.txt'
  return pd.read_csv(targets_txt, sep='\t')


2024-07-04 09:57:24.418832: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0


In [2]:
human = get_dataset('human', 'train', num_threads=1).batch(1)
#let's see how many batches there are
print(len(human)) #5313

2024-07-04 09:59:38.511412: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcuda.so.1
2024-07-04 09:59:39.036379: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1733] Found device 0 with properties: 
pciBusID: 0000:17:00.0 name: NVIDIA A40 computeCapability: 8.6
coreClock: 1.74GHz coreCount: 84 deviceMemorySize: 44.35GiB deviceMemoryBandwidth: 648.29GiB/s
2024-07-04 09:59:39.040411: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1733] Found device 1 with properties: 
pciBusID: 0000:65:00.0 name: NVIDIA A40 computeCapability: 8.6
coreClock: 1.74GHz coreCount: 84 deviceMemorySize: 44.35GiB deviceMemoryBandwidth: 648.29GiB/s
2024-07-04 09:59:39.042482: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1733] Found device 2 with properties: 
pciBusID: 0000:ca:00.0 name: NVIDIA A40 computeCapability: 8.6
coreClock: 1.74GHz coreCount: 84 deviceMemorySize: 44.35GiB deviceMemoryBandwidth: 648.29GiB/s
2024-07-04 09:59:39.046056: I t

TypeError: dataset length is unknown.

In [3]:
num_batches = sum(1 for _ in human)
print(f"Number of batches: {num_batches}")

2024-07-04 10:00:42.543156: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:176] None of the MLIR Optimization Passes are enabled (registered 2)
2024-07-04 10:00:42.544037: I tensorflow/core/platform/profile_utils/cpu_utils.cc:114] CPU Frequency: 2600000000 Hz


Number of batches: 34021


In [6]:
#expect this to be quite large! 34000 is indeed large, let's see the total space
a = next(iter(human))
a1 = a['sequence'].numpy()
a2 = a['target'].numpy()

In [7]:
#now print the size of a1 and a2
print(a1.shape)
print(a2.shape)
#and how much space they take
print(a1.nbytes/1e9)
print(a2.nbytes/1e9)

(1, 131072, 4)
(1, 896, 5313)
0.002097152
0.019041792


In [8]:
space = a1.nbytes+a2.nbytes
print(space/1e9)
print(space*34021/1e9) #yeah 720 GB is way too much lmfao! But if we tokenize it should be a lot less!

0.021138944
719.168013824


In [11]:
#so what we will do is tokenize one sequendce to see how long the sequence is when we store all of them! Can store it as int 8!
b = np.zeros(131072).astype(np.int8)
print((b.nbytes/a1.nbytes)*720) #only 6% as much!! 45 GB for the entire dataset just the sequences!!
#and let's see the targets which will likely be much larger...

45.0


In [12]:
#let's see the type
a2.dtype

dtype('float32')

In [15]:
#so we need to iterate over the dataset, and then we need to tokenize the sequence, and then save it
#we can do this with a for loop, and then we can save it as a numpy array
#so let's create a function that turns the one hot encoded to ACGTN
one_hot_to_base = {
    (1, 0, 0, 0): 7,
    (0, 1, 0, 0): 8,
    (0, 0, 1, 0): 9,
    (0, 0, 0, 1): 10,
    (0, 0, 0, 0): 11  # all zeros map to 'N'
}

# Function to convert one-hot encoded data to tokenized data
def one_hot_to_token(one_hot_data):
    # Reshape data if needed (assume input shape is (1, 131072, 4))
    one_hot_data = one_hot_data.reshape(-1, 4)
    
    # Convert each one-hot vector to a tuple to use as a dictionary key
    tokens = [one_hot_to_base[tuple(vector)] for vector in one_hot_data]
    
    return np.array(tokens, dtype=np.int8)

# Example one-hot encoded data
one_hot_data = a1
# Shape: (1, 5, 4) for demonstration

# Convert the data
tokenized_data = one_hot_to_token(one_hot_data)

print(tokenized_data)  # Output: "ACNGT"

[10  9  7 ... 10  7 10]


In [16]:
print(tokenized_data.shape) #yeah it all seems good!
print(tokenized_data[:15])
print(one_hot_data[0,:15])

(131072,)
[10  9  7 10 10  8 10 10  7  7  9  7 10 10 10]
[[0. 0. 0. 1.]
 [0. 0. 1. 0.]
 [1. 0. 0. 0.]
 [0. 0. 0. 1.]
 [0. 0. 0. 1.]
 [0. 1. 0. 0.]
 [0. 0. 0. 1.]
 [0. 0. 0. 1.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [0. 0. 1. 0.]
 [1. 0. 0. 0.]
 [0. 0. 0. 1.]
 [0. 0. 0. 1.]
 [0. 0. 0. 1.]]


In [18]:
print(tokenized_data.nbytes/a1.nbytes) #yeah it's the same thing! Each element is a byte!

0.0625


In [21]:
#but a2 is more complex
a2.max()

86.6875

In [3]:
#now let's loop it and save out the data

#we're gonna use HDF5! It's the best way to store large numpy arrays and we can store it paired
#let's test it tho

import glob
import json
import functools
import os
import tensorflow as tf
import numpy as np
import pandas as pd
from tqdm import tqdm

def organism_path(organism):
  return os.path.join('/data/leslie/sarthak/data/enformer/data/', organism) #just leads to the organism path, quite easy


def get_dataset(organism, subset, num_threads=8): #this gets the data and deserializes it
  metadata = get_metadata(organism)
  dataset = tf.data.TFRecordDataset(tfrecord_files(organism, subset),
                                    compression_type='ZLIB',
                                    num_parallel_reads=num_threads)
  dataset = dataset.map(functools.partial(deserialize, metadata=metadata),
                        num_parallel_calls=num_threads)
  return dataset


def get_metadata(organism):
  # Keys:
  # num_targets, train_seqs, valid_seqs, test_seqs, seq_length,
  # pool_width, crop_bp, target_length
  path = os.path.join(organism_path(organism), 'statistics.json')
  with tf.io.gfile.GFile(path, 'r') as f:
    return json.load(f)


def tfrecord_files(organism, subset): #this is the function that gets the TFR files, gets all of them and sorts them
  # Sort the values by int(*).
  return sorted(tf.io.gfile.glob(os.path.join(
      organism_path(organism), 'tfrecords', f'{subset}-*.tfr'
  )), key=lambda x: int(x.split('-')[-1].split('.')[0]))


def deserialize(serialized_example, metadata):
  """Deserialize bytes stored in TFRecordFile."""
  feature_map = {
      'sequence': tf.io.FixedLenFeature([], tf.string),
      'target': tf.io.FixedLenFeature([], tf.string),
  }
  example = tf.io.parse_example(serialized_example, feature_map)
  sequence = tf.io.decode_raw(example['sequence'], tf.bool)
  sequence = tf.reshape(sequence, (metadata['seq_length'], 4))
  sequence = tf.cast(sequence, tf.float32)

  target = tf.io.decode_raw(example['target'], tf.float16)
  target = tf.reshape(target,
                      (metadata['target_length'], metadata['num_targets']))
  target = tf.cast(target, tf.float32)

  return {'sequence': sequence,
          'target': target}

def get_targets(organism):
  targets_txt = f'https://raw.githubusercontent.com/calico/basenji/master/manuscripts/cross2020/targets_{organism}.txt'
  return pd.read_csv(targets_txt, sep='\t')

human = get_dataset('human', 'train', num_threads=1).batch(1)

one_hot_to_base = {
    (1, 0, 0, 0): 7,
    (0, 1, 0, 0): 8,
    (0, 0, 1, 0): 9,
    (0, 0, 0, 1): 10,
    (0, 0, 0, 0): 11  # all zeros map to 'N'
}

# Function to convert one-hot encoded data to tokenized data
def one_hot_to_token(one_hot_data):
    # Reshape data if needed (assume input shape is (1, 131072, 4))
    one_hot_data = one_hot_data.reshape(-1, 4)
    
    # Convert each one-hot vector to a tuple to use as a dictionary key
    tokens = [one_hot_to_base[tuple(vector)] for vector in one_hot_data]
    
    return np.array(tokens, dtype=np.int8)

#we need at least 50GB of ram to save it tho, so need to request a lot then we can do it!
sequence_array = np.zeros((34021,131072), dtype=np.int8)
print(sequence_array.nbytes/1e9) #oh only 4.5GB? That's quite good! in that case the other one at float32 should only be 4.5*8!
label_array = np.zeros((34021, 896, 513), dtype=np.float32)

4.459200512


In [4]:
test_array = np.zeros((34021,896,513), dtype=np.float32)
print(test_array.nbytes/1e9) #oh only 60 GB? That's quite good! in that case the other one at float32 should only be 4.5*8!
#lmao no it's 600 GB!!
#so we will save total_array as a int8 and then a numpy array, even .npy so no need to nonlazy load it, but then need to save the other one as h5py

62.550738432


In [5]:
sequence_array = np.zeros((34021,131072), dtype=np.int8)
for i, batch in tqdm(enumerate(human), total=34021):
    sequence = batch['sequence'].numpy()
    # target = batch['target'].numpy()
    tokenized_sequence = one_hot_to_token(sequence)
    sequence_array[i] = tokenized_sequence

  0%|          | 93/34021 [02:04<12:37:59,  1.34s/it]


: 

: 

In [1]:
#let's test to see how the human dataset works

#now let's loop it and save out the data

#we're gonna use HDF5! It's the best way to store large numpy arrays and we can store it paired
#let's test it tho

import glob
import json
import functools
import os
import tensorflow as tf
import numpy as np
import pandas as pd
from tqdm import tqdm

def organism_path(organism):
  return os.path.join('/data/leslie/sarthak/data/enformer/data/', organism) #just leads to the organism path, quite easy


def get_dataset(organism, subset, num_threads=8): #this gets the data and deserializes it
  metadata = get_metadata(organism)
  dataset = tf.data.TFRecordDataset(tfrecord_files(organism, subset),
                                    compression_type='ZLIB',
                                    num_parallel_reads=num_threads)
  dataset = dataset.map(functools.partial(deserialize, metadata=metadata),
                        num_parallel_calls=num_threads)
  return dataset


def get_metadata(organism):
  # Keys:
  # num_targets, train_seqs, valid_seqs, test_seqs, seq_length,
  # pool_width, crop_bp, target_length
  path = os.path.join(organism_path(organism), 'statistics.json')
  with tf.io.gfile.GFile(path, 'r') as f:
    return json.load(f)


def tfrecord_files(organism, subset): #this is the function that gets the TFR files, gets all of them and sorts them
  # Sort the values by int(*).
  return sorted(tf.io.gfile.glob(os.path.join(
      organism_path(organism), 'tfrecords', f'{subset}-*.tfr'
  )), key=lambda x: int(x.split('-')[-1].split('.')[0]))


def deserialize(serialized_example, metadata):
  """Deserialize bytes stored in TFRecordFile."""
  feature_map = {
      'sequence': tf.io.FixedLenFeature([], tf.string),
      'target': tf.io.FixedLenFeature([], tf.string),
  }
  example = tf.io.parse_example(serialized_example, feature_map)
  sequence = tf.io.decode_raw(example['sequence'], tf.bool)
  sequence = tf.reshape(sequence, (metadata['seq_length'], 4))
  sequence = tf.cast(sequence, tf.float32)

  target = tf.io.decode_raw(example['target'], tf.float16)
  target = tf.reshape(target,
                      (metadata['target_length'], metadata['num_targets']))
  target = tf.cast(target, tf.float32)

  return {'sequence': sequence,
          'target': target}

def get_targets(organism):
  targets_txt = f'https://raw.githubusercontent.com/calico/basenji/master/manuscripts/cross2020/targets_{organism}.txt'
  return pd.read_csv(targets_txt, sep='\t')

human = get_dataset('human', 'train', num_threads=1).batch(1)

one_hot_to_base = {
    (1, 0, 0, 0): 7,
    (0, 1, 0, 0): 8,
    (0, 0, 1, 0): 9,
    (0, 0, 0, 1): 10,
    (0, 0, 0, 0): 11  # all zeros map to 'N'
}

# Function to convert one-hot encoded data to tokenized data
def one_hot_to_token(one_hot_data):
    # Reshape data if needed (assume input shape is (1, 131072, 4))
    one_hot_data = one_hot_data.reshape(-1, 4)
    
    # Convert each one-hot vector to a tuple to use as a dictionary key
    tokens = [one_hot_to_base[tuple(vector)] for vector in one_hot_data]
    
    return np.array(tokens, dtype=np.int8)

#we need at least 50GB of ram to save it tho, so need to request a lot then we can do it!
sequence_array = np.zeros((34021,131072), dtype=np.int8)
print(sequence_array.nbytes/1e9) #oh only 4.5GB? That's quite good! in that case the other one at float32 should only be 4.5*8!
# label_array = np.zeros((34021, 896, 513), dtype=np.float32)

2024-07-04 16:48:47.569517: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
2024-07-04 16:49:28.548766: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcuda.so.1
2024-07-04 16:49:28.984000: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1733] Found device 0 with properties: 
pciBusID: 0000:17:00.0 name: NVIDIA A40 computeCapability: 8.6
coreClock: 1.74GHz coreCount: 84 deviceMemorySize: 44.35GiB deviceMemoryBandwidth: 648.29GiB/s
2024-07-04 16:49:28.987647: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1733] Found device 1 with properties: 
pciBusID: 0000:65:00.0 name: NVIDIA A40 computeCapability: 8.6
coreClock: 1.74GHz coreCount: 84 deviceMemorySize: 44.35GiB deviceMemoryBandwidth: 648.29GiB/s
2024-07-04 16:49:28.995177: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1733] Found device 2 with properties: 
pciBusID: 0000:ca:00.0 name: NVIDIA A40 com

4.459200512


In [2]:
it = iter(human)
small_array = np.zeros((5,896,5313))
for i in range(5):
    batch = next(it)
    label = batch['target'].numpy()
    small_array[i] = label

2024-07-04 12:02:29.288040: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:176] None of the MLIR Optimization Passes are enabled (registered 2)
2024-07-04 12:02:29.289686: I tensorflow/core/platform/profile_utils/cpu_utils.cc:114] CPU Frequency: 2300115000 Hz


In [6]:
label.shape

(1, 896, 5313)

In [4]:
small_array.max()

411.0

In [5]:
small_array

array([[[0.10839844, 0.10760498, 0.04425049, ..., 0.        ,
         0.        , 0.        ],
        [0.10162354, 0.09332275, 0.0094986 , ..., 0.        ,
         0.        , 0.        ],
        [0.10272217, 0.15600586, 0.        , ..., 0.        ,
         0.        , 0.        ],
        ...,
        [0.07714844, 0.07678223, 0.03509521, ..., 0.        ,
         0.01934814, 0.        ],
        [0.07666016, 0.03826904, 0.05648804, ..., 0.        ,
         0.        , 0.        ],
        [0.08319092, 0.06051636, 0.02156067, ..., 0.        ,
         0.        , 0.        ]],

       [[0.04019165, 0.06155396, 0.        , ..., 0.        ,
         0.        , 0.48095703],
        [0.13452148, 0.15014648, 0.        , ..., 0.        ,
         0.984375  , 0.        ],
        [0.17260742, 0.38916016, 0.02510071, ..., 0.        ,
         0.        , 0.        ],
        ...,
        [0.09771729, 0.07757568, 0.04217529, ..., 0.        ,
         0.48291016, 0.        ],
        [0.0

In [7]:
import h5py
#let's save this to test it
with h5py.File('/data/leslie/sarthak/data/enformer/data/test.h5', 'w') as f:
    f.create_dataset('label', data=small_array)

In [8]:
#now load it in
with h5py.File('/data/leslie/sarthak/data/enformer/data/test.h5', 'r') as f:
    print(f.keys())
    print(f['label'].shape)
    print(f['label'][:5])

<KeysViewHDF5 ['label']>
(5, 896, 5313)
[[[0.10839844 0.10760498 0.04425049 ... 0.         0.         0.        ]
  [0.10162354 0.09332275 0.0094986  ... 0.         0.         0.        ]
  [0.10272217 0.15600586 0.         ... 0.         0.         0.        ]
  ...
  [0.07714844 0.07678223 0.03509521 ... 0.         0.01934814 0.        ]
  [0.07666016 0.03826904 0.05648804 ... 0.         0.         0.        ]
  [0.08319092 0.06051636 0.02156067 ... 0.         0.         0.        ]]

 [[0.04019165 0.06155396 0.         ... 0.         0.         0.48095703]
  [0.13452148 0.15014648 0.         ... 0.         0.984375   0.        ]
  [0.17260742 0.38916016 0.02510071 ... 0.         0.         0.        ]
  ...
  [0.09771729 0.07757568 0.04217529 ... 0.         0.48291016 0.        ]
  [0.08557129 0.04833984 0.01190948 ... 0.         0.         0.        ]
  [0.0609436  0.08276367 0.11486816 ... 0.         0.         0.        ]]

 [[0.07098389 0.08081055 0.03701782 ... 0.         0.   

In [9]:
loaded_h5 = h5py.File('/data/leslie/sarthak/data/enformer/data/test.h5', 'r')
print(loaded_h5.keys())

<KeysViewHDF5 ['label']>


In [10]:
loaded_h5['label'].shape

(5, 896, 5313)

In [12]:
#let's calculate how large the full thing would be
size = 34021*896*5313 #then each of these is int32 so each one is 4 bytes
total = size*4
print(total/1e9) #so 650 GB, that's quite a lot, but it's doable

647.820805632


In [2]:
import numpy as np
import h5py
batch_size = 2
human = get_dataset('human', 'train', num_threads=1).batch(batch_size)
# Example data dimensions
filename = '/data/leslie/sarthak/data/enformer/data/test_large.h5'
# Function to create and save data incrementally
def create_hdf5_file(dataset, filename, chunk_size, compression='lzf'):
    # Create an HDF5 file
    with h5py.File(filename, 'w') as f:
        float32_data = f.create_dataset(
            'labels',
            shape=(34021, 896, 5313),
            maxshape=(34021, 896,5313),
            dtype='f4',  # float32 data type
            chunks=(chunk_size, 896, 5313),
            compression=compression
        )
        print('created dataset')
        print(float32_data.shape)
        idx=0
        for batch in dataset:
            label = batch['target'].numpy()
            print('loaded batch')
            float32_data[idx:idx+label.shape[0]] = label
            idx += label.shape[0]
            if idx>2:
                return idx



# Create HDF5 file with chunking and compression
create_hdf5_file(human, filename, batch_size)
#writing this is EXTREMELY slow!


created dataset
(34021, 896, 5313)


2024-07-04 16:25:43.244074: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:176] None of the MLIR Optimization Passes are enabled (registered 2)
2024-07-04 16:25:43.245135: I tensorflow/core/platform/profile_utils/cpu_utils.cc:114] CPU Frequency: 2300115000 Hz


loaded batch
loaded batch


4

In [4]:
#ran it for a bit, let's see it
import h5py
with h5py.File('/data/leslie/sarthak/data/enformer/data/test_large.h5', 'r') as f:
    print(f.keys())
    print(f['labels'].shape)
    print(f['labels'][:5])
    print(f['labels'].nbytes/1e9) #yeah it's quite quick!!


<KeysViewHDF5 ['labels']>
(34021, 896, 5313)
[[[0.10839844 0.10760498 0.04425049 ... 0.         0.         0.        ]
  [0.10162354 0.09332275 0.0094986  ... 0.         0.         0.        ]
  [0.10272217 0.15600586 0.         ... 0.         0.         0.        ]
  ...
  [0.07714844 0.07678223 0.03509521 ... 0.         0.01934814 0.        ]
  [0.07666016 0.03826904 0.05648804 ... 0.         0.         0.        ]
  [0.08319092 0.06051636 0.02156067 ... 0.         0.         0.        ]]

 [[0.04019165 0.06155396 0.         ... 0.         0.         0.48095703]
  [0.13452148 0.15014648 0.         ... 0.         0.984375   0.        ]
  [0.17260742 0.38916016 0.02510071 ... 0.         0.         0.        ]
  ...
  [0.09771729 0.07757568 0.04217529 ... 0.         0.48291016 0.        ]
  [0.08557129 0.04833984 0.01190948 ... 0.         0.         0.        ]
  [0.0609436  0.08276367 0.11486816 ... 0.         0.         0.        ]]

 [[0.07098389 0.08081055 0.03701782 ... 0.         

In [5]:
import numpy as np
import h5py
batch_size = 128
human = get_dataset('human', 'train', num_threads=1).batch(batch_size)
# Example data dimensions
filename = '/data/leslie/sarthak/data/enformer/data/train_label.h5'
# Function to create and save data incrementally
def create_hdf5_file(dataset, filename, chunk_size, compression='lzf'):
    # Create an HDF5 file
    with h5py.File(filename, 'w') as f:
        float32_data = f.create_dataset(
            'labels',
            shape=(34021, 896, 5313),
            maxshape=(34021, 896,5313),
            dtype='f4',  # float32 data type
            chunks=(chunk_size, 896, 5313),
            compression=compression
        )
        # print('created dataset')
        # print(float32_data.shape)
        idx=0
        for i,batch in tqdm(enumerate(dataset), total = 34021//batch_size):
            label = batch['target'].numpy()
            print('loaded batch')
            float32_data[idx:idx+label.shape[0]] = label
            idx += label.shape[0]
            if idx>2:
                return idx
#it's indeed quite slow, loading took a while and then actually doing it also took a while, 


# Create HDF5 file with chunking and compression
create_hdf5_file(human, filename, batch_size)
#writing this is EXTREMELY slow!
#1.5 minutes out of the 265... yeah it's fine tho

created dataset
(34021, 896, 5313)


  0%|          | 0/265 [00:00<?, ?it/s]

loaded batch


  0%|          | 0/265 [00:48<?, ?it/s]


128

In [6]:
#see the space 128 takes
print(128*896*5313*4/1e9) #so 2.5 GB, so 2.5 GB per batch, so 2.5*34021/1e9
#we can easily quadruple it and add more cores, 

2.437349376


In [7]:
100/2.5

#so let's do 32*128, maybe 16 to be safe... nah let's try 32

40.0

In [8]:
32*128 #batch size 4096, with 100 GB of ram, across 4 workers, so request 25 total

4096

In [2]:
human_val = get_dataset('human', 'valid', num_threads=1).batch(1)

In [4]:
next(iter(human_val))['sequence'].shape

TensorShape([1, 131072, 4])

In [5]:
#now iterate through it
for i, batch in enumerate(human_val):
    2
print(i)

2212


In [6]:
#number of elements is that + 1, because index is 1 lower
#so is 2213
human_test = get_dataset('human', 'test', num_threads=1).batch(1)
for i, batch in enumerate(human_test):
    2
print(i)

1936


# now let's compare some data from both

In [1]:
#let's test to see how the human dataset works

#now let's loop it and save out the data

#we're gonna use HDF5! It's the best way to store large numpy arrays and we can store it paired
#let's test it tho

import glob
import json
import functools
import os
import tensorflow as tf
import numpy as np
import pandas as pd
from tqdm import tqdm

def organism_path(organism):
  return os.path.join('/data/leslie/sarthak/data/enformer/data/', organism) #just leads to the organism path, quite easy


def get_dataset(organism, subset, num_threads=8): #this gets the data and deserializes it
  metadata = get_metadata(organism)
  dataset = tf.data.TFRecordDataset(tfrecord_files(organism, subset),
                                    compression_type='ZLIB',
                                    num_parallel_reads=num_threads)
  dataset = dataset.map(functools.partial(deserialize, metadata=metadata),
                        num_parallel_calls=num_threads)
  return dataset


def get_metadata(organism):
  # Keys:
  # num_targets, train_seqs, valid_seqs, test_seqs, seq_length,
  # pool_width, crop_bp, target_length
  path = os.path.join(organism_path(organism), 'statistics.json')
  with tf.io.gfile.GFile(path, 'r') as f:
    return json.load(f)


def tfrecord_files(organism, subset): #this is the function that gets the TFR files, gets all of them and sorts them
  # Sort the values by int(*).
  return sorted(tf.io.gfile.glob(os.path.join(
      organism_path(organism), 'tfrecords', f'{subset}-*.tfr'
  )), key=lambda x: int(x.split('-')[-1].split('.')[0]))


def deserialize(serialized_example, metadata):
  """Deserialize bytes stored in TFRecordFile."""
  feature_map = {
      'sequence': tf.io.FixedLenFeature([], tf.string),
      'target': tf.io.FixedLenFeature([], tf.string),
  }
  example = tf.io.parse_example(serialized_example, feature_map)
  sequence = tf.io.decode_raw(example['sequence'], tf.bool)
  sequence = tf.reshape(sequence, (metadata['seq_length'], 4))
  sequence = tf.cast(sequence, tf.float32)

  target = tf.io.decode_raw(example['target'], tf.float16)
  target = tf.reshape(target,
                      (metadata['target_length'], metadata['num_targets']))
  target = tf.cast(target, tf.float32)

  return {'sequence': sequence,
          'target': target}

def get_targets(organism):
  targets_txt = f'https://raw.githubusercontent.com/calico/basenji/master/manuscripts/cross2020/targets_{organism}.txt'
  return pd.read_csv(targets_txt, sep='\t')

human = get_dataset('human', 'train', num_threads=1).batch(1)

one_hot_to_base = {
    (1, 0, 0, 0): 7,
    (0, 1, 0, 0): 8,
    (0, 0, 1, 0): 9,
    (0, 0, 0, 1): 10,
    (0, 0, 0, 0): 11  # all zeros map to 'N'
}

# Function to convert one-hot encoded data to tokenized data
def one_hot_to_token(one_hot_data):
    # Reshape data if needed (assume input shape is (1, 131072, 4))
    one_hot_data = one_hot_data.reshape(-1, 4)
    
    # Convert each one-hot vector to a tuple to use as a dictionary key
    tokens = [one_hot_to_base[tuple(vector)] for vector in one_hot_data]
    
    return np.array(tokens, dtype=np.int8)

2024-07-05 11:18:37.033308: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
2024-07-05 11:18:56.825376: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcuda.so.1
2024-07-05 11:18:56.956668: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1733] Found device 0 with properties: 
pciBusID: 0000:02:00.0 name: NVIDIA GeForce GTX 1080 Ti computeCapability: 6.1
coreClock: 1.582GHz coreCount: 28 deviceMemorySize: 10.91GiB deviceMemoryBandwidth: 451.17GiB/s
2024-07-05 11:18:56.959075: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1733] Found device 1 with properties: 
pciBusID: 0000:03:00.0 name: NVIDIA GeForce GTX 1080 Ti computeCapability: 6.1
coreClock: 1.582GHz coreCount: 28 deviceMemorySize: 10.91GiB deviceMemoryBandwidth: 451.17GiB/s
2024-07-05 11:18:56.959638: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1733] Found device 2 with properties: 
pciBusID:

In [5]:
import h5py
train_label = h5py.File('/data/leslie/sarthak/data/enformer/data/train_label.h5', 'r')

In [5]:
#print the keys
train_label['labels'].shape

(34021, 896, 5313)

In [6]:
it = iter(human)
ex1 = next(it)
print(ex1)

2024-07-05 11:05:12.494884: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:176] None of the MLIR Optimization Passes are enabled (registered 2)
2024-07-05 11:05:12.496298: I tensorflow/core/platform/profile_utils/cpu_utils.cc:114] CPU Frequency: 2300040000 Hz


{'sequence': <tf.Tensor: shape=(1, 131072, 4), dtype=float32, numpy=
array([[[0., 0., 0., 1.],
        [0., 0., 1., 0.],
        [1., 0., 0., 0.],
        ...,
        [0., 0., 0., 1.],
        [1., 0., 0., 0.],
        [0., 0., 0., 1.]]], dtype=float32)>, 'target': <tf.Tensor: shape=(1, 896, 5313), dtype=float32, numpy=
array([[[0.10839844, 0.10760498, 0.04425049, ..., 0.        ,
         0.        , 0.        ],
        [0.10162354, 0.09332275, 0.0094986 , ..., 0.        ,
         0.        , 0.        ],
        [0.10272217, 0.15600586, 0.        , ..., 0.        ,
         0.        , 0.        ],
        ...,
        [0.07714844, 0.07678223, 0.03509521, ..., 0.        ,
         0.01934814, 0.        ],
        [0.07666016, 0.03826904, 0.05648804, ..., 0.        ,
         0.        , 0.        ],
        [0.08319092, 0.06051636, 0.02156067, ..., 0.        ,
         0.        , 0.        ]]], dtype=float32)>}


In [7]:
ex1['target'].shape

TensorShape([1, 896, 5313])

In [8]:
np.allclose(ex1['target'].numpy(), train_label['labels'][0])

True

In [9]:
data = train_label['labels'][0] #wait loading it is quite slow??
#so per epoch that adds 16 seconds... really??

In [17]:
#let's try a few more examples
human = get_dataset('human', 'train', num_threads=1).batch(32)
it = iter(human)
ex2 = next(it)
print(ex2)

{'sequence': <tf.Tensor: shape=(32, 131072, 4), dtype=float32, numpy=
array([[[0., 0., 0., 1.],
        [0., 0., 1., 0.],
        [1., 0., 0., 0.],
        ...,
        [0., 0., 0., 1.],
        [1., 0., 0., 0.],
        [0., 0., 0., 1.]],

       [[0., 0., 0., 1.],
        [1., 0., 0., 0.],
        [0., 0., 1., 0.],
        ...,
        [1., 0., 0., 0.],
        [0., 0., 1., 0.],
        [0., 0., 1., 0.]],

       [[1., 0., 0., 0.],
        [0., 0., 1., 0.],
        [0., 0., 1., 0.],
        ...,
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.]],

       ...,

       [[0., 0., 1., 0.],
        [0., 1., 0., 0.],
        [0., 0., 1., 0.],
        ...,
        [0., 0., 1., 0.],
        [0., 0., 1., 0.],
        [0., 0., 0., 1.]],

       [[0., 1., 0., 0.],
        [0., 0., 0., 1.],
        [0., 0., 1., 0.],
        ...,
        [0., 0., 1., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.]],

       [[1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1

In [7]:
#see this grabs it rather quick
data = train_label['labels'][0] #wait loading it is quite slow??
#now compare them
np.allclose(ex2['target'][0].numpy(), data)

True

In [8]:
data = train_label['labels'][1] #wait loading it is quite slow??
np.allclose(ex2['target'][1].numpy(), data)

False

In [10]:
print(ex2['target'][1])
print(data)
#oh it's actually quite different... why??

#some issue with the way it loads the batch being inconsistent??

tf.Tensor(
[[0.04019165 0.06155396 0.         ... 0.         0.         0.48095703]
 [0.13452148 0.15014648 0.         ... 0.         0.984375   0.        ]
 [0.17260742 0.38916016 0.02510071 ... 0.         0.         0.        ]
 ...
 [0.09771729 0.07757568 0.04217529 ... 0.         0.48291016 0.        ]
 [0.08557129 0.04833984 0.01190948 ... 0.         0.         0.        ]
 [0.0609436  0.08276367 0.11486816 ... 0.         0.         0.        ]], shape=(896, 5313), dtype=float32)
[[0.07519531 0.01112366 0.01609802 ... 0.         0.         0.        ]
 [0.0881958  0.01392365 0.03829956 ... 0.         0.         0.        ]
 [0.05108643 0.01622009 0.01737976 ... 0.         0.         0.        ]
 ...
 [0.00142574 0.001544   0.         ... 0.         0.         0.        ]
 [0.00675964 0.         0.         ... 0.         0.         0.        ]
 [0.05609131 0.0559082  0.06762695 ... 0.         0.         0.        ]]


In [15]:
print(ex2['sequence'][31])

tf.Tensor(
[[1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 ...
 [0. 0. 0. 1.]
 [0. 1. 0. 0.]
 [0. 0. 1. 0.]], shape=(131072, 4), dtype=float32)


In [21]:
#let's try just loading the TF data with a mapping

def organism_path(organism):
    return os.path.join('/data/leslie/sarthak/data/enformer/data/', organism)

def get_dataset(organism, subset, num_threads=8):
    metadata = get_metadata(organism)
    dataset = tf.data.TFRecordDataset(tfrecord_files(organism, subset),
                                      compression_type='ZLIB',
                                      num_parallel_reads=num_threads)
    dataset = dataset.map(functools.partial(deserialize, metadata=metadata),
                          num_parallel_calls=num_threads)
    return dataset

def get_metadata(organism):
    path = os.path.join(organism_path(organism), 'statistics.json')
    with tf.io.gfile.GFile(path, 'r') as f:
        return json.load(f)

def tfrecord_files(organism, subset):
    return sorted(tf.io.gfile.glob(os.path.join(
        organism_path(organism), 'tfrecords', f'{subset}-*.tfr'
    )), key=lambda x: int(x.split('-')[-1].split('.')[0]))

def deserialize(serialized_example, metadata):
    feature_map = {
        'sequence': tf.io.FixedLenFeature([], tf.string),
        'target': tf.io.FixedLenFeature([], tf.string),
    }
    example = tf.io.parse_example(serialized_example, feature_map)
    sequence = tf.io.decode_raw(example['sequence'], tf.bool)
    sequence = tf.reshape(sequence, (metadata['seq_length'], 4))
    sequence = tf.cast(sequence, tf.float32)

    target = tf.io.decode_raw(example['target'], tf.float16)
    target = tf.reshape(target,
                        (metadata['target_length'], metadata['num_targets']))
    target = tf.cast(target, tf.float32)

    return {'sequence': sequence, 'target': target}

def get_targets(organism):
    targets_txt = f'https://raw.githubusercontent.com/calico/basenji/master/manuscripts/cross2020/targets_{organism}.txt'
    return pd.read_csv(targets_txt, sep='\t')

def get_example_by_index(organism, subset, index, num_threads=8):
    metadata = get_metadata(organism)
    dataset = tf.data.TFRecordDataset(tfrecord_files(organism, subset),
                                      compression_type='ZLIB',
                                      num_parallel_reads=num_threads)
    dataset = dataset.map(functools.partial(deserialize, metadata=metadata),
                          num_parallel_calls=num_threads)
    iterator = iter(dataset.skip(index).take(1))
    example = next(iterator)
    return example

# Example usage:
organism = 'human'
subset = 'train'

# Count the number of examples
dataset = get_dataset(organism, subset, num_threads=1)
num_examples = 34021
print(f'Total number of examples: {num_examples}')

# Access the 32nd example (index 31)
example_index = 0
example = get_example_by_index(organism, subset, example_index)
print(f'Example at index {example_index}: {example}')

# Convert the example to numpy arrays
sequence_np = example['sequence'].numpy()
target_np = example['target'].numpy()
print(f'Sequence shape: {sequence_np.shape}, Target shape: {target_np.shape}')


Total number of examples: 34021
Example at index 0: {'sequence': <tf.Tensor: shape=(131072, 4), dtype=float32, numpy=
array([[0., 0., 0., 1.],
       [0., 0., 1., 0.],
       [1., 0., 0., 0.],
       ...,
       [0., 0., 0., 1.],
       [1., 0., 0., 0.],
       [0., 0., 0., 1.]], dtype=float32)>, 'target': <tf.Tensor: shape=(896, 5313), dtype=float32, numpy=
array([[0.10839844, 0.10760498, 0.04425049, ..., 0.        , 0.        ,
        0.        ],
       [0.10162354, 0.09332275, 0.0094986 , ..., 0.        , 0.        ,
        0.        ],
       [0.10272217, 0.15600586, 0.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.07714844, 0.07678223, 0.03509521, ..., 0.        , 0.01934814,
        0.        ],
       [0.07666016, 0.03826904, 0.05648804, ..., 0.        , 0.        ,
        0.        ],
       [0.08319092, 0.06051636, 0.02156067, ..., 0.        , 0.        ,
        0.        ]], dtype=float32)>}
Sequence shape: (131072, 4), Target shape: (8

In [22]:
#let's compare
print(ex2['sequence'][0])
print(sequence_np)

tf.Tensor(
[[0. 0. 0. 1.]
 [0. 0. 1. 0.]
 [1. 0. 0. 0.]
 ...
 [0. 0. 0. 1.]
 [1. 0. 0. 0.]
 [0. 0. 0. 1.]], shape=(131072, 4), dtype=float32)
[[0. 0. 0. 1.]
 [0. 0. 1. 0.]
 [1. 0. 0. 0.]
 ...
 [0. 0. 0. 1.]
 [1. 0. 0. 0.]
 [0. 0. 0. 1.]]


In [23]:
#that seems correct, now check out ex2 1
print(ex2['sequence'][1])
example = get_example_by_index(organism, subset, 1)
print(example['sequence'])

tf.Tensor(
[[0. 0. 0. 1.]
 [1. 0. 0. 0.]
 [0. 0. 1. 0.]
 ...
 [1. 0. 0. 0.]
 [0. 0. 1. 0.]
 [0. 0. 1. 0.]], shape=(131072, 4), dtype=float32)
tf.Tensor(
[[0. 0. 0. 1.]
 [0. 0. 0. 1.]
 [1. 0. 0. 0.]
 ...
 [0. 0. 0. 1.]
 [0. 0. 0. 1.]
 [1. 0. 0. 0.]], shape=(131072, 4), dtype=float32)


In [24]:
# I figured out the issue, iti seems when calling.batch, tensorflow shuffles the data, so it's not in the same order as the h5 file
human = get_dataset('human', 'train', num_threads=1).batch(1)
it = iter(human)
#now go through 5 iterations
for i in range(5):
    ex = next(it)
    print(ex['sequence'].shape)


(1, 131072, 4)
(1, 131072, 4)
(1, 131072, 4)
(1, 131072, 4)
(1, 131072, 4)


In [26]:
print(ex)

{'sequence': <tf.Tensor: shape=(1, 131072, 4), dtype=float32, numpy=
array([[[0., 0., 1., 0.],
        [0., 1., 0., 0.],
        [0., 0., 0., 1.],
        ...,
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [0., 0., 0., 1.]]], dtype=float32)>, 'target': <tf.Tensor: shape=(1, 896, 5313), dtype=float32, numpy=
array([[[0.01335907, 0.00794983, 0.        , ..., 0.        ,
         0.984375  , 0.        ],
        [0.00754929, 0.00812531, 0.        , ..., 0.        ,
         0.        , 0.        ],
        [0.00970459, 0.00933075, 0.        , ..., 0.        ,
         0.        , 0.        ],
        ...,
        [0.10101318, 0.06243896, 0.3190918 , ..., 0.        ,
         0.        , 0.        ],
        [0.05557251, 0.00617599, 0.05374146, ..., 0.        ,
         0.        , 0.01580811],
        [0.07263184, 0.02262878, 0.02156067, ..., 0.        ,
         0.        , 0.        ]]], dtype=float32)>}


In [27]:
example = get_example_by_index(organism, subset, 4)
print(example)

{'sequence': <tf.Tensor: shape=(131072, 4), dtype=float32, numpy=
array([[0., 1., 0., 0.],
       [0., 1., 0., 0.],
       [0., 1., 0., 0.],
       ...,
       [0., 1., 0., 0.],
       [1., 0., 0., 0.],
       [1., 0., 0., 0.]], dtype=float32)>, 'target': <tf.Tensor: shape=(896, 5313), dtype=float32, numpy=
array([[0.06286621, 0.03756714, 0.00804901, ..., 0.        , 0.        ,
        0.        ],
       [0.09143066, 0.0609436 , 0.04440308, ..., 0.        , 0.        ,
        0.        ],
       [0.09613037, 0.05172729, 0.01834106, ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.03494263, 0.07391357, 0.00627518, ..., 0.        , 0.        ,
        0.        ],
       [0.02326965, 0.0553894 , 0.09417725, ..., 0.        , 0.        ,
        0.        ],
       [0.02053833, 0.04690552, 0.22131348, ..., 0.        , 0.        ,
        0.        ]], dtype=float32)>}


In [25]:
#now we have the index 4 data, let's compare this
data = train_label['labels'][4]
np.allclose(example['target'].numpy(), data)
np.allclose(ex['target'].numpy(), data)

False

In [35]:
#hmmmmm, it could be that it happens when we load the dataset, let's use the same dataset
human_dataset = get_dataset('human', 'train', num_threads=1)
human_1 = human_dataset.batch(1)
human_2 = human_dataset.batch(2)
it = iter(human_1)
#let's get the second example
ex = next(it)
ex2 = next(it)
print(ex2['sequence'])

tf.Tensor(
[[[0. 0. 0. 1.]
  [1. 0. 0. 0.]
  [0. 0. 1. 0.]
  ...
  [1. 0. 0. 0.]
  [0. 0. 1. 0.]
  [0. 0. 1. 0.]]], shape=(1, 131072, 4), dtype=float32)


In [36]:
#now try this human 2
it = iter(human_2)
ex = next(it)
print(ex['sequence'][1]) #ok so the batching is identical, the issue seems to be if we call the dataset again

tf.Tensor(
[[0. 0. 0. 1.]
 [1. 0. 0. 0.]
 [0. 0. 1. 0.]
 ...
 [1. 0. 0. 0.]
 [0. 0. 1. 0.]
 [0. 0. 1. 0.]], shape=(131072, 4), dtype=float32)


In [37]:
human_dataset = get_dataset('human', 'train', num_threads=1)
human_1 = human_dataset.batch(1)
human_2 = human_dataset.batch(2)
it = iter(human_1)
#let's get the second example
ex = next(it)
ex2 = next(it)
print(ex2['sequence'])
#this is consistent, and it seems to be exactly the same!

tf.Tensor(
[[[0. 0. 0. 1.]
  [1. 0. 0. 0.]
  [0. 0. 1. 0.]
  ...
  [1. 0. 0. 0.]
  [0. 0. 1. 0.]
  [0. 0. 1. 0.]]], shape=(1, 131072, 4), dtype=float32)


In [ ]:
#no, again this is exactly the same???

In [40]:
#So I think the issue just has to do with batching the data for some reason
example = get_example_by_index(organism, subset, 1)
print(example)

{'sequence': <tf.Tensor: shape=(131072, 4), dtype=float32, numpy=
array([[0., 0., 0., 1.],
       [0., 0., 0., 1.],
       [1., 0., 0., 0.],
       ...,
       [0., 0., 0., 1.],
       [0., 0., 0., 1.],
       [1., 0., 0., 0.]], dtype=float32)>, 'target': <tf.Tensor: shape=(896, 5313), dtype=float32, numpy=
array([[0.07519531, 0.01112366, 0.01609802, ..., 0.        , 0.        ,
        0.        ],
       [0.0881958 , 0.01392365, 0.03829956, ..., 0.        , 0.        ,
        0.        ],
       [0.05108643, 0.01622009, 0.01737976, ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.00142574, 0.001544  , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.00675964, 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.05609131, 0.0559082 , 0.06762695, ..., 0.        , 0.        ,
        0.        ]], dtype=float32)>}


# redo functions and define it with a set dataset

In [2]:
#let's try just loading the TF data with a mapping
import os
import json
import functools
import tensorflow as tf
import numpy as np

def organism_path(organism):
    return os.path.join('/data/leslie/sarthak/data/enformer/data/', organism)

def get_dataset(organism, subset, num_threads=8):
    metadata = get_metadata(organism)
    dataset = tf.data.TFRecordDataset(tfrecord_files(organism, subset),
                                      compression_type='ZLIB',
                                      num_parallel_reads=num_threads)
    dataset = dataset.map(functools.partial(deserialize, metadata=metadata),
                          num_parallel_calls=num_threads)
    return dataset

def get_metadata(organism):
    path = os.path.join(organism_path(organism), 'statistics.json')
    with tf.io.gfile.GFile(path, 'r') as f:
        return json.load(f)

def tfrecord_files(organism, subset):
    return sorted(tf.io.gfile.glob(os.path.join(
        organism_path(organism), 'tfrecords', f'{subset}-*.tfr'
    )), key=lambda x: int(x.split('-')[-1].split('.')[0]))

def deserialize(serialized_example, metadata):
    feature_map = {
        'sequence': tf.io.FixedLenFeature([], tf.string),
        'target': tf.io.FixedLenFeature([], tf.string),
    }
    example = tf.io.parse_example(serialized_example, feature_map)
    sequence = tf.io.decode_raw(example['sequence'], tf.bool)
    sequence = tf.reshape(sequence, (metadata['seq_length'], 4))
    sequence = tf.cast(sequence, tf.float32)

    target = tf.io.decode_raw(example['target'], tf.float16)
    target = tf.reshape(target,
                        (metadata['target_length'], metadata['num_targets']))
    target = tf.cast(target, tf.float32)

    return {'sequence': sequence, 'target': target}

def get_targets(organism):
    targets_txt = f'https://raw.githubusercontent.com/calico/basenji/master/manuscripts/cross2020/targets_{organism}.txt'
    return pd.read_csv(targets_txt, sep='\t')

def get_example_by_index(dataset, organism, subset, index, num_threads=8):
    metadata = get_metadata(organism)
    # dataset = tf.data.TFRecordDataset(tfrecord_files(organism, subset),
    #                                   compression_type='ZLIB',
    #                                   num_parallel_reads=num_threads)
    # dataset = dataset.map(functools.partial(deserialize, metadata=metadata),
    #                       num_parallel_calls=num_threads)
    iterator = iter(dataset.skip(index).take(1))
    example = next(iterator)
    return example

# Example usage:
organism = 'human'
subset = 'train'

# Count the number of examples
dataset = get_dataset(organism, subset, num_threads=1)
num_examples = 34021
# print(f'Total number of examples: {num_examples}')

# Access the 2nd example (index 1)
example_index = 1
example = get_example_by_index(dataset, organism, subset, example_index)
print(f'Example at index {example_index}: {example}')

2024-07-05 12:00:04.382955: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
2024-07-05 12:00:22.872394: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcuda.so.1
2024-07-05 12:00:23.029781: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1733] Found device 0 with properties: 
pciBusID: 0000:02:00.0 name: NVIDIA GeForce GTX 1080 Ti computeCapability: 6.1
coreClock: 1.582GHz coreCount: 28 deviceMemorySize: 10.91GiB deviceMemoryBandwidth: 451.17GiB/s
2024-07-05 12:00:23.033005: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1733] Found device 1 with properties: 
pciBusID: 0000:03:00.0 name: NVIDIA GeForce GTX 1080 Ti computeCapability: 6.1
coreClock: 1.582GHz coreCount: 28 deviceMemorySize: 10.91GiB deviceMemoryBandwidth: 451.17GiB/s
2024-07-05 12:00:23.033572: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1733] Found device 2 with properties: 
pciBusID:

Example at index 1: {'sequence': <tf.Tensor: shape=(131072, 4), dtype=float32, numpy=
array([[0., 0., 0., 1.],
       [1., 0., 0., 0.],
       [0., 0., 1., 0.],
       ...,
       [1., 0., 0., 0.],
       [0., 0., 1., 0.],
       [0., 0., 1., 0.]], dtype=float32)>, 'target': <tf.Tensor: shape=(896, 5313), dtype=float32, numpy=
array([[0.04019165, 0.06155396, 0.        , ..., 0.        , 0.        ,
        0.48095703],
       [0.13452148, 0.15014648, 0.        , ..., 0.        , 0.984375  ,
        0.        ],
       [0.17260742, 0.38916016, 0.02510071, ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.09771729, 0.07757568, 0.04217529, ..., 0.        , 0.48291016,
        0.        ],
       [0.08557129, 0.04833984, 0.01190948, ..., 0.        , 0.        ,
        0.        ],
       [0.0609436 , 0.08276367, 0.11486816, ..., 0.        , 0.        ,
        0.        ]], dtype=float32)>}


In [45]:
human1 = dataset.batch(1)
it = iter(human1)
ex1 = next(it)
ex2 = next(it)
print(ex2['sequence'])
print(ex2['target'])

tf.Tensor(
[[[0. 0. 0. 1.]
  [1. 0. 0. 0.]
  [0. 0. 1. 0.]
  ...
  [1. 0. 0. 0.]
  [0. 0. 1. 0.]
  [0. 0. 1. 0.]]], shape=(1, 131072, 4), dtype=float32)
tf.Tensor(
[[[0.04019165 0.06155396 0.         ... 0.         0.         0.48095703]
  [0.13452148 0.15014648 0.         ... 0.         0.984375   0.        ]
  [0.17260742 0.38916016 0.02510071 ... 0.         0.         0.        ]
  ...
  [0.09771729 0.07757568 0.04217529 ... 0.         0.48291016 0.        ]
  [0.08557129 0.04833984 0.01190948 ... 0.         0.         0.        ]
  [0.0609436  0.08276367 0.11486816 ... 0.         0.         0.        ]]], shape=(1, 896, 5313), dtype=float32)


In [44]:
human2 = dataset.batch(2)
it = iter(human2)
ex1 = next(it)
print(ex1['sequence'][1])
print(ex1['target'][1])

tf.Tensor(
[[0. 0. 0. 1.]
 [1. 0. 0. 0.]
 [0. 0. 1. 0.]
 ...
 [1. 0. 0. 0.]
 [0. 0. 1. 0.]
 [0. 0. 1. 0.]], shape=(131072, 4), dtype=float32)
tf.Tensor(
[[0.04019165 0.06155396 0.         ... 0.         0.         0.48095703]
 [0.13452148 0.15014648 0.         ... 0.         0.984375   0.        ]
 [0.17260742 0.38916016 0.02510071 ... 0.         0.         0.        ]
 ...
 [0.09771729 0.07757568 0.04217529 ... 0.         0.48291016 0.        ]
 [0.08557129 0.04833984 0.01190948 ... 0.         0.         0.        ]
 [0.0609436  0.08276367 0.11486816 ... 0.         0.         0.        ]], shape=(896, 5313), dtype=float32)


In [6]:
#these look identical! So as long as we use the same dataset, it should be the same, let's test one more
dataset = get_dataset(organism, subset, num_threads=1)
example = get_example_by_index(dataset, organism, subset, 10) #oof, it's actually really slow...
print(example)

{'sequence': <tf.Tensor: shape=(131072, 4), dtype=float32, numpy=
array([[0., 0., 0., 1.],
       [0., 0., 0., 1.],
       [0., 0., 0., 1.],
       ...,
       [0., 0., 1., 0.],
       [0., 0., 0., 1.],
       [0., 0., 0., 1.]], dtype=float32)>, 'target': <tf.Tensor: shape=(896, 5313), dtype=float32, numpy=
array([[0.07965088, 0.07824707, 0.05599976, ..., 0.        , 0.        ,
        0.00358963],
       [0.08392334, 0.15319824, 0.10559082, ..., 0.        , 0.        ,
        0.30615234],
       [0.0760498 , 0.07672119, 0.09527588, ..., 0.        , 0.01780701,
        0.        ],
       ...,
       [0.03396606, 0.03102112, 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.0748291 , 0.02720642, 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.07141113, 0.0362854 , 0.09655762, ..., 0.        , 0.        ,
        0.        ]], dtype=float32)>}


In [5]:
import h5py
train_label = h5py.File('/data/leslie/sarthak/data/enformer/data/train_label.h5', 'r')
data = train_label['labels'][10]
print(data)

[[0.09265137 0.19274902 0.265625   ... 0.         0.         0.        ]
 [0.12792969 0.16992188 0.4387207  ... 0.         0.         0.        ]
 [0.12878418 0.14343262 0.44189453 ... 0.         0.         0.12255859]
 ...
 [0.06341553 0.08654785 0.27294922 ... 0.         0.         0.        ]
 [0.15966797 0.12335205 0.45141602 ... 0.9736328  0.         0.        ]
 [0.14868164 0.10693359 0.27368164 ... 0.         0.         0.        ]]


In [16]:
#let's do some further testing
num_threads = 1
batch_size = 2
num_elems = 34021
chunk_size = 2
compression = 'lzf'
import numpy as np
import h5py
human = get_dataset('human', 'train', num_threads=num_threads).batch(batch_size)
# Example data dimensions
filename = '/data/leslie/sarthak/data/enformer/data/test.h5'
# Function to create and save data incrementally
batch = next(iter(human))
print(batch['target'].shape)

(2, 896, 5313)


In [18]:
from tqdm import tqdm
with h5py.File(filename, 'w') as f:
    float32_data = f.create_dataset(
        'labels',
        shape=(num_elems, 896, 5313),
        maxshape=(num_elems, 896,5313),
        dtype='f4',  # float32 data type
        chunks=(chunk_size, 896, 5313),
        compression=compression,
    )
    # print('created dataset')
    # print(float32_data.shape)
    idx=0
    for i,batch in tqdm(enumerate(dataset), total=num_elems//batch_size):
        label = batch['target'].numpy()
        print('loaded batch')
        float32_data[idx:idx+label.shape[0]] = label
        idx += label.shape[0]
        if i==5:
            break
print(idx)

  0%|          | 0/17010 [00:00<?, ?it/s]

loaded batch


  0%|          | 1/17010 [03:27<982:42:13, 207.99s/it]

loaded batch


  0%|          | 2/17010 [07:12<1028:41:16, 217.74s/it]

...

loaded batch


  0%|          | 5/17010 [18:10<1032:26:31, 218.57s/it]

loaded batch


  0%|          | 5/17010 [21:39<1227:32:49, 259.87s/it]


5376


In [23]:
#now let's view it!
batch['target'].shape #oh it was messed up because used dataset which was not batched, assigned iti to 896 different ones lol

TensorShape([896, 5313])

In [24]:
for i,batch in tqdm(enumerate(human), total=num_elems//batch_size):
    label = batch['target'].numpy()
    print(label.shape)

  0%|          | 1/17010 [00:00<1:30:44,  3.12it/s]

(2, 896, 5313)


  0%|          | 3/17010 [00:00<59:47,  4.74it/s]  

(2, 896, 5313)
(2, 896, 5313)


  0%|          | 5/17010 [00:00<49:44,  5.70it/s]

...

(2, 896, 5313)
(2, 896, 5313)


  0%|          | 17/17010 [00:02<41:25,  6.84it/s]

(2, 896, 5313)
(2, 896, 5313)


  0%|          | 17/17010 [00:02<46:59,  6.03it/s]


KeyboardInterrupt: 

In [25]:
batch_size

2

In [13]:
load_data = h5py.File(filename, 'r')
data = load_data['labels'][10]
print(data)
load_data.close()

[[0.07965088 0.07824707 0.05599976 ... 0.         0.         0.00358963]
 [0.08392334 0.15319824 0.10559082 ... 0.         0.         0.30615234]
 [0.0760498  0.07672119 0.09527588 ... 0.         0.01780701 0.        ]
 ...
 [0.03396606 0.03102112 0.         ... 0.         0.         0.        ]
 [0.0748291  0.02720642 0.         ... 0.         0.         0.        ]
 [0.07141113 0.0362854  0.09655762 ... 0.         0.         0.        ]]


In [10]:
print(batch['target'].numpy()[10])

[[0.07965088 0.07824707 0.05599976 ... 0.         0.         0.00358963]
 [0.08392334 0.15319824 0.10559082 ... 0.         0.         0.30615234]
 [0.0760498  0.07672119 0.09527588 ... 0.         0.01780701 0.        ]
 ...
 [0.03396606 0.03102112 0.         ... 0.         0.         0.        ]
 [0.0748291  0.02720642 0.         ... 0.         0.         0.        ]
 [0.07141113 0.0362854  0.09655762 ... 0.         0.         0.        ]]


In [11]:
#but let's compare thaht with the full dataset
load_data2 = h5py.File('/data/leslie/sarthak/data/enformer/data/train_label.h5', 'r')
data2 = load_data2['labels'][10]
print(data2)

[[0.09265137 0.19274902 0.265625   ... 0.         0.         0.        ]
 [0.12792969 0.16992188 0.4387207  ... 0.         0.         0.        ]
 [0.12878418 0.14343262 0.44189453 ... 0.         0.         0.12255859]
 ...
 [0.06341553 0.08654785 0.27294922 ... 0.         0.         0.        ]
 [0.15966797 0.12335205 0.45141602 ... 0.9736328  0.         0.        ]
 [0.14868164 0.10693359 0.27368164 ... 0.         0.         0.        ]]


In [26]:
#finally let's just load in the real one and see if the bottom row is zero or not
loaded = h5py.File('/data/leslie/sarthak/data/enformer/data/train_label.h5', 'r')
print(loaded['labels'].shape)

(34021, 896, 5313)


In [27]:
loaded['labels'][-1] #has values, let's just assume it's correct

array([[0.01947021, 0.03265381, 0.01818848, ..., 0.        , 0.04885864,
        0.        ],
       [0.01475525, 0.03619385, 0.00531006, ..., 0.        , 0.        ,
        0.        ],
       [0.02165222, 0.03610229, 0.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.06347656, 0.03421021, 0.10302734, ..., 0.14416504, 1.2871094 ,
        0.58691406],
       [0.05636597, 0.03067017, 0.01673889, ..., 0.        , 0.        ,
        0.        ],
       [0.0335083 , 0.05386353, 0.01303864, ..., 0.        , 0.        ,
        0.        ]], dtype=float32)

# testing sequence

In [28]:
#let's see if the sequences are correct
seq = np.load('/data/leslie/sarthak/data/enformer/data/train_sequence.npy')

In [29]:
print(seq.shape)

(34021, 131072)


In [33]:
print(seq[0,:20])
print(seq[10,:20])
#TGATTCTTAAGATTTTCCAG
#CAGTCTCCCTGTCTACGAAT

[10  9  7 10 10  8 10 10  7  7  9  7 10 10 10 10  8  8  7  9]
[ 8  7  9 10  8 10  8  8  8 10  9 10  8 10  7  8  9  7  7 10]


In [39]:
#now let's see if the sequences are correct
human = get_dataset('human', 'train', num_threads=1).batch(1)
it = iter(human)
ex1 = next(it)
print(ex1['sequence'][0,:20])
#TGATTCTTAAGATTTTCCAG
#perfectly aligns!! at least for the first one

tf.Tensor(
[[0. 0. 0. 1.]
 [0. 0. 1. 0.]
 [1. 0. 0. 0.]
 [0. 0. 0. 1.]
 [0. 0. 0. 1.]
 [0. 1. 0. 0.]
 [0. 0. 0. 1.]
 [0. 0. 0. 1.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [0. 0. 1. 0.]
 [1. 0. 0. 0.]
 [0. 0. 0. 1.]
 [0. 0. 0. 1.]
 [0. 0. 0. 1.]
 [0. 0. 0. 1.]
 [0. 1. 0. 0.]
 [0. 1. 0. 0.]
 [1. 0. 0. 0.]
 [0. 0. 1. 0.]], shape=(20, 4), dtype=float32)


In [41]:
human = get_dataset('human', 'train', num_threads=1).batch(10)
it = iter(human)
ex1 = next(it)
print(ex1['sequence'][9,:20])
#AGGAAAGATTATTCCACGAT
#a fundamentally different sequence, this isn't even RC...

tf.Tensor(
[[1. 0. 0. 0.]
 [0. 0. 1. 0.]
 [0. 0. 1. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [0. 0. 1. 0.]
 [1. 0. 0. 0.]
 [0. 0. 0. 1.]
 [0. 0. 0. 1.]
 [1. 0. 0. 0.]
 [0. 0. 0. 1.]
 [0. 0. 0. 1.]
 [0. 1. 0. 0.]
 [0. 1. 0. 0.]
 [1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 1. 0.]
 [1. 0. 0. 0.]
 [0. 0. 0. 1.]], shape=(20, 4), dtype=float32)


In [48]:
human = get_dataset('human', 'train', num_threads=1).batch(1)
it = iter(human)
for i in range(10):
    ex = next(it)
print(ex['sequence'].shape)
print(ex['sequence'][0,:20])
#so thisi s the same as that dataset, huh I guess maybe the order does shift around??
#the other big difference might have to do with the workers

(1, 131072, 4)
tf.Tensor(
[[1. 0. 0. 0.]
 [0. 0. 1. 0.]
 [0. 0. 1. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [0. 0. 1. 0.]
 [1. 0. 0. 0.]
 [0. 0. 0. 1.]
 [0. 0. 0. 1.]
 [1. 0. 0. 0.]
 [0. 0. 0. 1.]
 [0. 0. 0. 1.]
 [0. 1. 0. 0.]
 [0. 1. 0. 0.]
 [1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 1. 0.]
 [1. 0. 0. 0.]
 [0. 0. 0. 1.]], shape=(20, 4), dtype=float32)


In [43]:
896*5313/131072
#so 36x as much data in the labels.

36.3193359375

In [58]:
def tfrecord_files(organism, subset): #this is the function that gets the TFR files, gets all of them and sorts them
  # Sort the values by int(*).
  return sorted(tf.io.gfile.glob(os.path.join(
      organism_path(organism), 'tfrecords', f'{subset}-*.tfr'
  )), key=lambda x: int(x.split('-')[-1].split('.')[0]))
a = tfrecord_files('human', 'train')
print(a)
#so this is always loaded identically with the sorted
#I'm quite confident it's due to the multiple workers and how the dataset handles that

['/data/leslie/sarthak/data/enformer/data/human/tfrecords/train-0-0.tfr', '/data/leslie/sarthak/data/enformer/data/human/tfrecords/train-0-1.tfr', '/data/leslie/sarthak/data/enformer/data/human/tfrecords/train-0-2.tfr', '/data/leslie/sarthak/data/enformer/data/human/tfrecords/train-0-3.tfr', '/data/leslie/sarthak/data/enformer/data/human/tfrecords/train-0-4.tfr', '/data/leslie/sarthak/data/enformer/data/human/tfrecords/train-0-5.tfr', '/data/leslie/sarthak/data/enformer/data/human/tfrecords/train-0-6.tfr', '/data/leslie/sarthak/data/enformer/data/human/tfrecords/train-0-7.tfr', '/data/leslie/sarthak/data/enformer/data/human/tfrecords/train-0-8.tfr', '/data/leslie/sarthak/data/enformer/data/human/tfrecords/train-0-9.tfr', '/data/leslie/sarthak/data/enformer/data/human/tfrecords/train-0-10.tfr', '/data/leslie/sarthak/data/enformer/data/human/tfrecords/train-0-11.tfr', '/data/leslie/sarthak/data/enformer/data/human/tfrecords/train-0-12.tfr', '/data/leslie/sarthak/data/enformer/data/human/

# testing the loss

In [3]:
import tensorflow as tf
import numpy as np

def poisson(yt, yp, epsilon: float = 1e-7):
    """Poisson loss, without mean reduction."""
    return yp - yt * tf.math.log(yp + epsilon)


def poisson_multinomial(
    y_true,
    y_pred,
    total_weight: float = 1,
    weight_range: float = 1,
    weight_exp: int = 4,
    epsilon: float = 1e-7,
    rescale: bool = False,
):
    """Possion decomposition with multinomial specificity term.

    Args:
        total_weight (float): Weight of the Poisson total term.
        epsilon (float): Added small value to avoid log(0).
        rescale (bool): Rescale loss after re-weighting.
    """
    seq_len = y_true.shape[1]
    
    if weight_range < 1:
        raise ValueError("Poisson Multinomial weight_range must be >=1")
    elif weight_range == 1:
        position_weights = tf.ones((1, seq_len, 1))
    else:
        pos_start = -(seq_len / 2 - 0.5)
        pos_end = seq_len / 2 + 0.5
        positions = tf.range(pos_start, pos_end, dtype=tf.float32)
        sigma = -pos_start / (np.log(weight_range)) ** (1 / weight_exp)
        position_weights = tf.exp(-((positions / sigma) ** weight_exp))
        position_weights /= tf.reduce_max(position_weights)
        position_weights = tf.expand_dims(position_weights, axis=0)
        position_weights = tf.expand_dims(position_weights, axis=-1)

    y_true = tf.math.multiply(y_true, position_weights)
    y_pred = tf.math.multiply(y_pred, position_weights)

    # sum across lengths
    s_true = tf.math.reduce_sum(y_true, axis=-2) # B x T
    s_pred = tf.math.reduce_sum(y_pred, axis=-2) # B x T

    # total count poisson loss, mean across targets
    poisson_term = poisson(s_true, s_pred)  # B x T
    poisson_term /= tf.reduce_sum(position_weights)

    # add epsilon to protect against tiny values
    y_true += epsilon
    y_pred += epsilon

    # normalize to sum to one
    p_pred = y_pred / tf.expand_dims(s_pred, axis=-2) # B x L x T

    # multinomial loss
    pl_pred = tf.math.log(p_pred)  # B x L x T
    multinomial_dot = -tf.math.multiply(y_true, pl_pred)  # B x L x T
    multinomial_term = tf.math.reduce_sum(multinomial_dot, axis=-2)  # B x T
    multinomial_term /= tf.reduce_sum(position_weights)

    # normalize to scale of 1:1 term ratio
    loss_raw = multinomial_term + total_weight * poisson_term
    if rescale:
        loss_rescale = loss_raw * 2 / (1 + total_weight)
    else:
        loss_rescale = loss_raw

    return loss_rescale

In [9]:
#let's test this position weight stuff
weight_range = 1.5
seq_len = 10
weight_exp =4

pos_start = -(seq_len / 2 - 0.5)
pos_end = seq_len / 2 + 0.5
positions = tf.range(pos_start, pos_end, dtype=tf.float32)
sigma = -pos_start / (np.log(weight_range)) ** (1 / weight_exp)
position_weights = tf.exp(-((positions / sigma) ** weight_exp))
position_weights /= tf.reduce_max(position_weights)
position_weights = tf.expand_dims(position_weights, axis=0)
position_weights = tf.expand_dims(position_weights, axis=-1)

print(position_weights.shape)
print(position_weights) #so weights the center more... that does make a lot of sense
#and up to 1.5* as much in the middle as compared to the ends... hmmm I think we'll just go with the same as the paper

(1, 10, 1)
tf.Tensor(
[[[0.6667078]
  [0.8621566]
  [0.9621714]
  [0.9950683]
  [1.       ]
  [1.       ]
  [0.9950683]
  [0.9621714]
  [0.8621566]
  [0.6667078]]], shape=(1, 10, 1), dtype=float32)


In [13]:
#the thing is I don't believe that's used for the loss, I think they just use poisson
loss_fn = tf.keras.losses.Poisson(reduction=tf.keras.losses.Reduction.NONE)
#loss = y_pred - y_true * log(y_pred)
#let's run a test data through the loss function
#set the numpy random seed
np.random.seed(0)
yt = np.random.rand(1, 896, 5313)
yp = np.random.rand(1, 896, 5313)
print(yt)

[[[0.5488135  0.71518937 0.60276338 ... 0.63212081 0.14788355 0.95568239]
  [0.12785748 0.77676417 0.31861386 ... 0.30325158 0.26912988 0.70332816]
  [0.15640513 0.84344823 0.86062903 ... 0.03735818 0.22248574 0.85637566]
  ...
  [0.89456813 0.59438585 0.66791604 ... 0.05151961 0.91356269 0.55993623]
  [0.43623324 0.95413652 0.44569275 ... 0.63626726 0.32920214 0.24306976]
  [0.36453648 0.68097476 0.07672555 ... 0.62856675 0.25360279 0.65144222]]]


In [19]:
#now print it out
print(a:=loss_fn(yp, yt)) #the shape is perfect, as that is the loss per class, so we need to keep them separate for the multitasking, then later sum, but can see how it performs here

tf.Tensor(
[[1.00022292 1.00757444 1.00415909 1.00359941 1.00718081 1.00416183
  0.99344254 0.99909258 1.00300491 0.99306333 0.99845058 0.99022168
  1.00071287 0.99707371 0.98563379 1.00084507 0.9928121  1.00180829
  1.00850582 1.00211418 1.00247204 0.99590069 1.00041306 1.00224841
  1.01315427 0.9951458  1.00442684 1.00265813 1.00241566 0.99757701
  0.99688488 1.00116313 0.99127781 0.9946245  0.99067163 0.99931455
  0.99310327 1.00716162 1.00698757 0.99767929 0.9996357  0.99766952
  1.01239705 0.99424928 1.00188434 0.99451548 1.00510573 0.99444735
  1.00790846 0.9991039  1.00330055 0.99709892 0.99819225 1.00282311
  1.00023794 1.00487578 0.99372268 0.99318743 1.00604045 1.00781834
  1.01526725 0.99130601 1.00461531 1.00587463 1.00496805 0.99951851
  0.99620724 1.00721371 0.99886042 1.00199795 0.98976231 0.9955222
  1.00958121 1.00602722 0.98738235 0.99414504 1.00160146 0.9888891
  0.99438679 1.0021646  0.98880106 1.00092995 0.9960779  1.00324845
  0.99835968 1.00442517 1.00486374 0.99

In [18]:
#find mean of a
print(np.mean(a))

1.0000541438348591


In [ ]:
import torch
import torch.nn as nn
class PoissonLoss(nn.Module):
    def __init__(self, reduction='none'):
        super(PoissonLoss, self).__init__()
        self.reduction = reduction
    def forward(self, y_pred, y_true):
        loss = y_pred - y_true * torch.log(y_pred)
        if self.reduction == 'mean':
            return torch.mean(loss)
        elif self.reduction == 'sum':
            return torch.sum(loss)
        else:
            return loss

#let's test this
yt = torch.tensor(yt)
yp = torch.tensor(yp)

loss_fn = PoissonLoss(reduction='none')
loss = loss_fn(yp,yt)


In [ ]:
#let's see it for a specific output

import tensorflow as tf
import tensorflow_hub as hub

enformer_model = hub.load("https://kaggle.com/models/deepmind/enformer/frameworks/TensorFlow2/variations/enformer/versions/1").model

SEQ_LENGTH = 393_216

# Numpy array [batch_size, SEQ_LENGTH, 4] one hot encoded in order 'ACGT'. The
# `one_hot_encode` function is available in `enformer.py` and outputs can be
# stacked to form a batch.
enformer_model
inputs = tf.zeros((1, SEQ_LENGTH, 4), dtype=tf.float32)
predictions = enformer_model.predict_on_batch(inputs)
predictions['human'].shape